# 21. SHAPモデル解釈 - 説明可能なAI (SHAP Model Interpretation)

## 概要
SHAPを使って機械学習モデルの予測を解釈し、説明可能にする方法を学びます。

## 学習目標
- SHAPの基本概念を理解できる
- 特徴量の重要度を定量化できる
- 個別予測の説明ができる
- グローバルな解釈とローカルな解釈の違いを理解できる
- 実務でSHAPを活用できる

In [ ]:
# 必要なライブラリのインポート
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import load_breast_cancer, fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, GradientBoostingRegressor
from sklearn.linear_model import LogisticRegression
import shap
import warnings
warnings.filterwarnings('ignore')

# 設定
plt.rcParams['font.sans-serif'] = ['DejaVu Sans']
plt.rcParams['axes.unicode_minus'] = False
np.random.seed(42)

# SHAP JavaScript の初期化（ノートブック用）
shap.initjs()

## 1. なぜモデルの解釈が必要か

### ブラックボックス問題

複雑な機械学習モデル（Random Forest、Gradient Boosting、Neural Networksなど）は高い予測精度を持つ一方、**なぜその予測をしたのか**が分かりにくいという問題があります。

### モデル解釈が重要な理由

1. **信頼性の向上**
   - 予測根拠を理解できる
   - バイアスや誤りを検出できる

2. **規制対応**
   - 金融、医療などの分野で説明責任が求められる
   - GDPR（EU一般データ保護規則）など

3. **モデル改善**
   - 重要な特徴量を特定
   - データ収集の優先順位決定

4. **ビジネス価値**
   - 意思決定者への説明
   - アクションプランの策定

## 2. SHAPとは

### SHAP (SHapley Additive exPlanations)

ゲーム理論のShapley値に基づいた、統一的なモデル解釈手法です。

### SHAPの特徴

1. **モデル非依存**
   - あらゆる機械学習モデルに適用可能
   - 統一的な解釈フレームワーク

2. **理論的保証**
   - 数学的に厳密な基盤
   - 公平な寄与度計算

3. **ローカル＆グローバル解釈**
   - 個別予測の説明
   - 全体的なパターンの理解

### SHAP値の意味

各特徴量が予測値にどれだけ貢献したかを示す値です。

- **正のSHAP値**: 予測を増加させる方向に寄与
- **負のSHAP値**: 予測を減少させる方向に寄与
- **SHAP値の合計**: ベースライン予測からの差分

## 3. 分類問題でのSHAP

### データ準備

In [ ]:
# 乳がんデータセット
cancer = load_breast_cancer()
X = pd.DataFrame(cancer.data, columns=cancer.feature_names)
y = cancer.target

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"データサイズ: {X.shape}")
print(f"\n特徴量の一部:")
print(X.columns[:10].tolist())

In [ ]:
# モデルの学習
model_rf = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42)
model_rf.fit(X_train, y_train)

print(f"訓練精度: {model_rf.score(X_train, y_train):.4f}")
print(f"テスト精度: {model_rf.score(X_test, y_test):.4f}")

### SHAP値の計算

In [ ]:
# TreeExplainer（ツリーベースモデル用の高速explainer）
explainer = shap.TreeExplainer(model_rf)

# テストデータのSHAP値を計算
shap_values = explainer.shap_values(X_test)

print(f"SHAP値の形状: {shap_values[1].shape}")
print(f"クラス数: {len(shap_values)}")
print(f"\nベースライン値（期待値）: {explainer.expected_value}")

### 4. グローバル解釈: 特徴量の重要度

全データに対する各特徴量の平均的な影響を見ます。

In [ ]:
# Summary Plot: 特徴量重要度と値の分布
shap.summary_plot(shap_values[1], X_test, plot_type="bar", show=False)
plt.title('Feature Importance (Mean |SHAP value|)')
plt.tight_layout()
plt.show()

print("このグラフの読み方:")
print("- 上位の特徴量ほど予測に大きく影響")
print("- 平均絶対SHAP値で順位付け")

In [ ]:
# Beeswarm Plot: 詳細な分布
shap.summary_plot(shap_values[1], X_test, show=False)
plt.title('SHAP Summary Plot (Beeswarm)')
plt.tight_layout()
plt.show()

print("\nBeeswarm Plotの読み方:")
print("- 各点が1つのサンプル")
print("- 色: 特徴量の値（赤=高、青=低）")
print("- X軸: SHAP値（予測への影響）")
print("- Y軸: 特徴量（重要度順）")

### 5. ローカル解釈: 個別予測の説明

特定のサンプルに対する予測の理由を詳しく見ます。

In [ ]:
# 1つのサンプルを選択
sample_idx = 0
sample = X_test.iloc[sample_idx]

# 予測
prediction = model_rf.predict([sample])[0]
probability = model_rf.predict_proba([sample])[0]

print(f"サンプル #{sample_idx}")
print(f"予測クラス: {cancer.target_names[prediction]}")
print(f"予測確率: {probability}")
print(f"実際のクラス: {cancer.target_names[y_test.iloc[sample_idx]]}")

In [ ]:
# Force Plot: ウォーターフォール式の説明
shap.force_plot(
    explainer.expected_value[1],
    shap_values[1][sample_idx],
    X_test.iloc[sample_idx],
    matplotlib=True,
    show=False
)
plt.title(f'SHAP Force Plot for Sample #{sample_idx}')
plt.tight_layout()
plt.show()

print("\nForce Plotの読み方:")
print("- 基準値（expected value）から予測値への変化を示す")
print("- 赤: 陽性方向に押す特徴量")
print("- 青: 陰性方向に押す特徴量")
print("- 幅: 影響の大きさ")

In [ ]:
# Waterfall Plot: より詳細な表示
shap.waterfall_plot(
    shap.Explanation(
        values=shap_values[1][sample_idx],
        base_values=explainer.expected_value[1],
        data=X_test.iloc[sample_idx].values,
        feature_names=X_test.columns.tolist()
    ),
    show=False
)
plt.title(f'SHAP Waterfall Plot for Sample #{sample_idx}')
plt.tight_layout()
plt.show()

### 6. 依存関係プロット

特定の特徴量がどのように予測に影響するかを詳しく見ます。

In [ ]:
# 最も重要な特徴量を選択
feature_name = 'worst perimeter'

# Dependence Plot
shap.dependence_plot(
    feature_name,
    shap_values[1],
    X_test,
    show=False
)
plt.title(f'SHAP Dependence Plot: {feature_name}')
plt.tight_layout()
plt.show()

print("\nDependence Plotの読み方:")
print("- X軸: 特徴量の値")
print("- Y軸: SHAP値（予測への影響）")
print("- 色: 他の重要な特徴量の値（相互作用）")
print("- 傾向: 特徴量と予測の関係性")

## 7. 回帰問題でのSHAP

In [ ]:
# カリフォルニア住宅価格データセット
housing = fetch_california_housing()
X_reg = pd.DataFrame(housing.data[:1000], columns=housing.feature_names)
y_reg = housing.target[:1000]

X_train_reg, X_test_reg, y_train_reg, y_test_reg = train_test_split(
    X_reg, y_reg, test_size=0.2, random_state=42
)

# モデル学習
model_gbr = GradientBoostingRegressor(n_estimators=100, max_depth=5, random_state=42)
model_gbr.fit(X_train_reg, y_train_reg)

print(f"訓練R²: {model_gbr.score(X_train_reg, y_train_reg):.4f}")
print(f"テストR²: {model_gbr.score(X_test_reg, y_test_reg):.4f}")

In [ ]:
# SHAP値の計算
explainer_reg = shap.TreeExplainer(model_gbr)
shap_values_reg = explainer_reg.shap_values(X_test_reg)

# 特徴量重要度
shap.summary_plot(shap_values_reg, X_test_reg, plot_type="bar", show=False)
plt.title('Feature Importance for Housing Price Prediction')
plt.tight_layout()
plt.show()

In [ ]:
# 詳細なSummary Plot
shap.summary_plot(shap_values_reg, X_test_reg, show=False)
plt.title('SHAP Summary for Housing Price')
plt.tight_layout()
plt.show()

print("\n解釈例:")
print("- MedInc（所得）が高いほど価格が上昇（正のSHAP値）")
print("- Latitude/Longitudeの影響（地理的要因）")
print("- 特徴量間の相互作用も可視化")

## 8. 実務での活用

### ユースケース別の使い分け

In [ ]:
def explain_prediction(model, explainer, sample, feature_names, target_names=None):
    """
    予測結果を詳しく説明する関数
    
    Parameters:
    -----------
    model : 学習済みモデル
    explainer : SHAP explainer
    sample : 説明したいサンプル
    feature_names : 特徴量名のリスト
    target_names : クラス名のリスト（分類の場合）
    """
    # 予測
    prediction = model.predict([sample])[0]
    
    # SHAP値
    shap_val = explainer.shap_values([sample])
    
    print("=" * 60)
    print("予測結果の説明")
    print("=" * 60)
    
    if target_names:
        # 分類
        proba = model.predict_proba([sample])[0]
        print(f"予測クラス: {target_names[prediction]}")
        print(f"予測確率: {proba[prediction]:.4f}")
        
        # 上位寄与特徴量
        shap_abs = np.abs(shap_val[1][0])
    else:
        # 回帰
        print(f"予測値: {prediction:.4f}")
        shap_abs = np.abs(shap_val[0])
    
    # 重要な特徴量トップ5
    top_indices = np.argsort(shap_abs)[-5:][::-1]
    
    print("\n影響の大きい特徴量 TOP 5:")
    for i, idx in enumerate(top_indices, 1):
        if target_names:
            shap_value = shap_val[1][0][idx]
        else:
            shap_value = shap_val[0][idx]
        
        direction = "↑" if shap_value > 0 else "↓"
        print(f"{i}. {feature_names[idx]}: {sample[idx]:.2f}")
        print(f"   SHAP値: {shap_value:.4f} {direction}")
    
    print("=" * 60)

# 使用例
explain_prediction(
    model_rf, 
    explainer, 
    X_test.iloc[0].values,
    X_test.columns.tolist(),
    cancer.target_names
)

## 9. まとめ

### 本ノートブックで学んだこと

1. **モデル解釈の重要性**
   - ブラックボックス問題
   - 信頼性、規制対応、ビジネス価値

2. **SHAPの基礎**
   - Shapley値に基づく統一的手法
   - 理論的保証と実用性

3. **グローバル解釈**
   - 特徴量重要度
   - Summary plot
   - 全体的なパターン理解

4. **ローカル解釈**
   - 個別予測の説明
   - Force plot、Waterfall plot
   - 予測根拠の可視化

5. **依存関係分析**
   - Dependence plot
   - 特徴量と予測の関係
   - 相互作用の発見

6. **実務での活用**
   - 分類・回帰両方での適用
   - カスタム説明関数
   - レポート自動生成

### SHAPを使うべきとき

- ✅ モデルの信頼性を高めたい
- ✅ ステークホルダーへの説明が必要
- ✅ 規制対応が求められる
- ✅ 特徴量エンジニアリングの改善
- ✅ バイアスや誤りの検出

### 次のステップ

- Notebook 22でStackingアンサンブルを学ぶ
- 実際のプロジェクトでSHAPを活用
- より高度な解釈手法を探求（LIME、Integrated Gradientsなど）